In [ ]:
import pandas as pd
import math
global Threshold,Epsilon,ErrTemp,TopFeature,Parity

FeatureData= pd.read_csv('FeatureData.csv')
FeatureData = FeatureData.drop(FeatureData.columns[[0]], axis=1)

shape = FeatureData.shape 

FaceImageListSize=shape[0]
F5Count=shape[1]-2
        
RoundAccuracy=[0]*10
RoundFP=[0]*10
RoundFN=[0]*10
BoostRounds=10
Epsilon=[0]*10
Alpha  =[0]*10
Beta   =[0]*10
Threshold =[0]*10
TopFeature =[0]*10
Polarity =[0]*10
ErrorFinal=[2]*10

BRound=0

Label =list(FeatureData[str(F5Count)].values)
Weight=list(FeatureData[str(F5Count+1)].values)
SortedData=pd.DataFrame()

SortedData.insert(loc=0,column=0,value=Label)
SortedData.insert(loc=1,column=1,value=Label)
SortedData.insert(loc=2,column=2,value=Weight)

##Adaboost with decision stumps
 
for BRound in range(BoostRounds):    
    print("Start Round:",BRound)
    FwtSum=0
    for i in range(FaceImageListSize):
        FwtSum=FeatureData.iat[i,(F5Count+1)]+FwtSum
    for f in range(FaceImageListSize):
        FeatureData.iat[f,F5Count+1]=((FeatureData.iat[f,F5Count+1])/FwtSum)

    SortedData[2]=FeatureData[str(F5Count+1)]            

    for FRound in range(F5Count-1):    
        j=str(FRound)
        SortedData[0]=FeatureData[j] 
        SortedData=SortedData.sort_values(by=0)
        Weight_1Pos=0
        Weight_1Neg=0

        Weight_m1Pos=0
        Weight_m1Neg=0
        
        ErrTemp=0
        Thresh=(SortedData.iat[0,0])-1

        for e in range(FaceImageListSize):     
            if(SortedData.iat[e,1]==1):
                Weight_1Pos=Weight_1Pos+SortedData.iat[e,2]                
            else:
                Weight_m1Pos=Weight_m1Pos+SortedData.iat[e,2]    
        imgcnt=0      
        while(1):           
            ErrPos=Weight_1Neg+Weight_m1Pos
            ErrNeg=Weight_1Pos+Weight_m1Neg      
            
            if(ErrPos<=ErrNeg):
                ErrTemp=ErrPos
                ParTemp=1
            else:
                ErrTemp=ErrNeg
                ParTemp=-1
                         
            if(ErrTemp<=ErrorFinal[BRound]):
                ErrorFinal[BRound]=ErrTemp
                Threshold[BRound]=Thresh
                TopFeature[BRound]=FRound
                Polarity[BRound]=ParTemp
            
            if(imgcnt==FaceImageListSize-1):
                break
            imgcnt=imgcnt+1
            
            while(1):
                if(SortedData.iat[imgcnt,1]==-1):
                    Weight_m1Neg=Weight_m1Neg+SortedData.iat[imgcnt,2]
                    Weight_m1Pos=Weight_m1Pos-SortedData.iat[imgcnt,2]
                else:
                    Weight_1Neg=Weight_1Neg+SortedData.iat[imgcnt,2]
                    Weight_1Pos=Weight_1Pos-SortedData.iat[imgcnt,2]
                    
                if((imgcnt==FaceImageListSize-1)or(SortedData.iat[imgcnt,0]!=SortedData.iat[imgcnt+1,0])):
                    break
                else:
                    imgcnt=imgcnt+1
            if(imgcnt==FaceImageListSize-1):
                Thresh=SortedData.iat[imgcnt,0]+1
            else:
                Thresh=(SortedData.iat[imgcnt,0]+SortedData.iat[imgcnt+1,0])/2    
    i=0
    hlabel1=1
    
    for i in range(FaceImageListSize):       
        if((FeatureData.iat[i,TopFeature[BRound]])>=(Threshold[BRound])):
            val=1
        else:
            val=-1     
        hlabel1=(val*Polarity[BRound])
        
        if(hlabel1!=FeatureData.iat[i,F5Count]):
            Epsilon[BRound]=Epsilon[BRound]+FeatureData.iat[i,(F5Count+1)]
  
    Eph=(1/(Epsilon[BRound]))
    EphInv=1/(1-Epsilon[BRound])    
    Alpha[BRound]=(1/2)*(math.log((1-Epsilon[BRound])/(Epsilon[BRound])))
    i=0
    hlabel2=1
    for i in range(FaceImageListSize):        
        if((FeatureData.iat[i,TopFeature[BRound]])>=(Threshold[BRound])):
            val=1
        else:
            val=-1     

        hlabel2=(val*Polarity[BRound])       
        if(hlabel2!=FeatureData.iat[i,F5Count]):
            FeatureData.iat[i,(F5Count+1)]=(1/2)*(FeatureData.iat[i,(F5Count+1)]*Eph)
        else:
            FeatureData.iat[i,(F5Count+1)]=(1/2)*(FeatureData.iat[i,(F5Count+1)]*(EphInv))

for Rcount in range(1,BoostRounds+1):
    for im in range(FaceImageListSize):
        HSum=0
        for b in range(Rcount):          
            if((FeatureData.iat[im,TopFeature[b]])>=(Threshold[b])):
                val=1
            else:
                val=-1        
            HSum=(Alpha[b]*val*Polarity[b])+HSum                     
                     
        if(HSum>=0):
            FinalDectection=1
        else:
            FinalDectection=-1
            
        if(FinalDectection!=FeatureData.iat[im,F5Count]):
            if(FinalDectection==1):
                RoundFP[Rcount-1]+=1
            else:
                RoundFN[Rcount-1]+=1
        else:
            RoundAccuracy[Rcount-1]+=1
    
    RoundFP[Rcount-1]=(RoundFP[Rcount-1]/FaceImageListSize)
    RoundFN[Rcount-1]=(RoundFN[Rcount-1]/FaceImageListSize)
    RoundAccuracy[Rcount-1]=(RoundAccuracy[Rcount-1]/FaceImageListSize)
    
print("Data for Empirical Error")        
print("Threshold is :",Threshold)
print("TopFeature is :",TopFeature)
print("RoundAccuracy is :",RoundAccuracy)
print("RoundFP is :",RoundFP)
print("RoundFN is :",RoundFN)


##Adaboost for False Positive
global FalsePos,FalseNeg,Acc
FeatureData= pd.read_csv('FeatureData.csv')
FeatureData = FeatureData.drop(FeatureData.columns[[0]], axis=1)

shape = FeatureData.shape 

FaceImageListSize=shape[0]
F5Count=shape[1]-2
        
RoundAccuracy=[0]*10
RoundFP=[0]*10
RoundFN=[0]*10
BoostRounds=10
Epsilon=[0]*10
Alpha  =[0]*10
Beta   =[0]*10
Threshold =[0]*10
TopFeature =[0]*10
Polarity =[0]*10
ErrorFinal=[2]*10

BRound=0

Label =list(FeatureData[str(F5Count)].values)
Weight=list(FeatureData[str(F5Count+1)].values)
SortedData=pd.DataFrame()

SortedData.insert(loc=0,column=0,value=Label)
SortedData.insert(loc=1,column=1,value=Label)
SortedData.insert(loc=2,column=2,value=Weight)
 
for BRound in range(BoostRounds):    
    print("Start Round:",BRound)
    FwtSum=0
    for i in range(FaceImageListSize):
        FwtSum=FeatureData.iat[i,(F5Count+1)]+FwtSum
    for f in range(FaceImageListSize):
        FeatureData.iat[f,F5Count+1]=((FeatureData.iat[f,F5Count+1])/FwtSum)

    SortedData[2]=FeatureData[str(F5Count+1)]            

    for FRound in range(F5Count-1):    
        j=str(FRound)
        SortedData[0]=FeatureData[j] 
        SortedData=SortedData.sort_values(by=0)
        Weight_1Pos=0
        Weight_1Neg=0

        Weight_m1Pos=0
        Weight_m1Neg=0
        
        ErrTemp=0
        Thresh=(SortedData.iat[0,0])-1

        for e in range(FaceImageListSize):     
            if(SortedData.iat[e,1]==1):
                Weight_1Pos=Weight_1Pos+SortedData.iat[e,2]                
            else:
                Weight_m1Pos=Weight_m1Pos+SortedData.iat[e,2]    
        imgcnt=0      
        while(1):           
            ErrPos=Weight_1Neg+Weight_m1Pos
            ErrNeg=Weight_1Pos+Weight_m1Neg      
            
            if(ErrPos<=ErrNeg):
                ErrTemp=ErrPos
                ParTemp=1
            else:
                ErrTemp=ErrNeg
                ParTemp=-1
                         
            if(ErrTemp<=ErrorFinal[BRound]):
                ErrorFinal[BRound]=ErrTemp
                Threshold[BRound]=Thresh
                TopFeature[BRound]=FRound
                Polarity[BRound]=ParTemp
            
            if(imgcnt==FaceImageListSize-1):
                break
            imgcnt=imgcnt+1
            
            while(1):
                if(SortedData.iat[imgcnt,1]==-1):
                    Weight_m1Neg=Weight_m1Neg+SortedData.iat[imgcnt,2]
                    Weight_m1Pos=Weight_m1Pos-SortedData.iat[imgcnt,2]
                else:
                    Weight_1Neg=Weight_1Neg+SortedData.iat[imgcnt,2]
                    Weight_1Pos=Weight_1Pos-SortedData.iat[imgcnt,2]
                    
                if((imgcnt==FaceImageListSize-1)or(SortedData.iat[imgcnt,0]!=SortedData.iat[imgcnt+1,0])):
                    break
                else:
                    imgcnt=imgcnt+1
            if(imgcnt==FaceImageListSize-1):
                Thresh=SortedData.iat[imgcnt,0]+1
            else:
                Thresh=(SortedData.iat[imgcnt,0]+SortedData.iat[imgcnt+1,0])/2    
    i=0
    hlabel1=1
    FalsePos=0
    FalseNeg=0
    Acc=0 
    
    FalsePosCount=0
    FalseNegCount=0
    Total=0 
    for i in range(FaceImageListSize):       
        if((FeatureData.iat[i,TopFeature[BRound]])>=(Threshold[BRound])):
            val=1
        else:
            val=-1     
        hlabel1=(val*Polarity[BRound])
        
        if(hlabel1!=FeatureData.iat[i,F5Count]):
            Epsilon[BRound]=Epsilon[BRound]+FeatureData.iat[i,(F5Count+1)]
            if(hlabel1==1):
                FalsePos=FalsePos+FeatureData.iat[i,(F5Count+1)]
                FalsePosCount+=1
            else:
                FalseNeg=FalseNeg+FeatureData.iat[i,(F5Count+1)]
                FalsePosCount+=1
     
    Total=FalsePosCount+FalsePosCount
    Coeff=FalsePosCount/Total
           
    Eph=(1/(Epsilon[BRound]))
    EphInv=1/(1-Epsilon[BRound])    
    Alpha[BRound]=(1/2)*(math.log((1-Epsilon[BRound])/(Epsilon[BRound])))
    i=0
    hlabel2=1
    
    for i in range(FaceImageListSize):        
        if((FeatureData.iat[i,TopFeature[BRound]])>=(Threshold[BRound])):
            val=1
        else:
            val=-1     

        hlabel2=(val*Polarity[BRound])       
        if(hlabel2!=FeatureData.iat[i,F5Count]):
            if(hlabel2==1):
                FeatureData.iat[i,(F5Count+1)]=(1/2)*(FeatureData.iat[i,(F5Count+1)]*Eph)*(1/Coeff)
            else:
                FeatureData.iat[i,(F5Count+1)]=(1/2)*(FeatureData.iat[i,(F5Count+1)]*Eph)
        else:
            FeatureData.iat[i,(F5Count+1)]=(1/2)*(FeatureData.iat[i,(F5Count+1)]*(EphInv))
            
for Rcount in range(1,BoostRounds+1):
    for im in range(FaceImageListSize):
        HSum=0
        for b in range(Rcount):          
            if((FeatureData.iat[im,TopFeature[b]])>=(Threshold[b])):
                val=1
            else:
                val=-1        
            HSum=(Alpha[b]*val*Polarity[b])+HSum                     
                     
        if(HSum>=0):
            FinalDectection=1
        else:
            FinalDectection=-1
            
        if(FinalDectection!=FeatureData.iat[im,F5Count]):
            if(FinalDectection==1):
                RoundFP[Rcount-1]+=1
            else:
                RoundFN[Rcount-1]+=1
        else:
            RoundAccuracy[Rcount-1]+=1
    
    RoundFP[Rcount-1]=(RoundFP[Rcount-1]/FaceImageListSize)
    RoundFN[Rcount-1]=(RoundFN[Rcount-1]/FaceImageListSize)
    RoundAccuracy[Rcount-1]=(RoundAccuracy[Rcount-1]/FaceImageListSize)
print("Data for False Positive")       
print("Threshold is :",Threshold)
print("TopFeature is :",TopFeature)
print("RoundAccuracy is :",RoundAccuracy)
print("RoundFP is :",RoundFP)
print("RoundFN is :",RoundFN)

##Adaboost for False Negative
FalsePos = 0
FalseNeg = 0
Acc = 0
FeatureData= pd.read_csv('FeatureData.csv')
FeatureData = FeatureData.drop(FeatureData.columns[[0]], axis=1)

shape = FeatureData.shape 

FaceImageListSize=shape[0]
F5Count=shape[1]-2
        
RoundAccuracy=[0]*10
RoundFP=[0]*10
RoundFN=[0]*10
BoostRounds=10
Epsilon=[0]*10
Alpha  =[0]*10
Beta   =[0]*10
Threshold =[0]*10
TopFeature =[0]*10
Polarity =[0]*10
ErrorFinal=[2]*10

BRound=0

Label =list(FeatureData[str(F5Count)].values)
Weight=list(FeatureData[str(F5Count+1)].values)
SortedData=pd.DataFrame()

SortedData.insert(loc=0,column=0,value=Label)
SortedData.insert(loc=1,column=1,value=Label)
SortedData.insert(loc=2,column=2,value=Weight)
 
for BRound in range(BoostRounds):    
    print("Start Round:",BRound)
    FwtSum=0
    for i in range(FaceImageListSize):
        FwtSum=FeatureData.iat[i,(F5Count+1)]+FwtSum
    for f in range(FaceImageListSize):
        FeatureData.iat[f,F5Count+1]=((FeatureData.iat[f,F5Count+1])/FwtSum)

    SortedData[2]=FeatureData[str(F5Count+1)]            

    for FRound in range(F5Count-1):    
        j=str(FRound)
        SortedData[0]=FeatureData[j] 
        SortedData=SortedData.sort_values(by=0)
        Weight_1Pos=0
        Weight_1Neg=0

        Weight_m1Pos=0
        Weight_m1Neg=0
        
        ErrTemp=0
        Thresh=(SortedData.iat[0,0])-1

        for e in range(FaceImageListSize):     
            if(SortedData.iat[e,1]==1):
                Weight_1Pos=Weight_1Pos+SortedData.iat[e,2]                
            else:
                Weight_m1Pos=Weight_m1Pos+SortedData.iat[e,2]    
        imgcnt=0      
        while(1):           
            ErrPos=Weight_1Neg+Weight_m1Pos
            ErrNeg=Weight_1Pos+Weight_m1Neg      
            
            if(ErrPos<=ErrNeg):
                ErrTemp=ErrPos
                ParTemp=1
            else:
                ErrTemp=ErrNeg
                ParTemp=-1
                         
            if(ErrTemp<=ErrorFinal[BRound]):
                ErrorFinal[BRound]=ErrTemp
                Threshold[BRound]=Thresh
                TopFeature[BRound]=FRound
                Polarity[BRound]=ParTemp
            
            if(imgcnt==FaceImageListSize-1):
                break
            imgcnt=imgcnt+1
            
            while(1):
                if(SortedData.iat[imgcnt,1]==-1):
                    Weight_m1Neg=Weight_m1Neg+SortedData.iat[imgcnt,2]
                    Weight_m1Pos=Weight_m1Pos-SortedData.iat[imgcnt,2]
                else:
                    Weight_1Neg=Weight_1Neg+SortedData.iat[imgcnt,2]
                    Weight_1Pos=Weight_1Pos-SortedData.iat[imgcnt,2]
                    
                if((imgcnt==FaceImageListSize-1)or(SortedData.iat[imgcnt,0]!=SortedData.iat[imgcnt+1,0])):
                    break
                else:
                    imgcnt=imgcnt+1
            if(imgcnt==FaceImageListSize-1):
                Thresh=SortedData.iat[imgcnt,0]+1
            else:
                Thresh=(SortedData.iat[imgcnt,0]+SortedData.iat[imgcnt+1,0])/2    
    i=0
    hlabel1=1
    FalsePos=0
    FalseNeg=0
    Acc=0 
    
    FalsePosCount=0
    FalseNegCount=0
    Total=0 
    for i in range(FaceImageListSize):       
        if((FeatureData.iat[i,TopFeature[BRound]])>=(Threshold[BRound])):
            val=1
        else:
            val=-1     
        hlabel1=(val*Polarity[BRound])
        
        if(hlabel1!=FeatureData.iat[i,F5Count]):
            Epsilon[BRound]=Epsilon[BRound]+FeatureData.iat[i,(F5Count+1)]
            if(hlabel1==1):
                FalsePos=FalsePos+FeatureData.iat[i,(F5Count+1)]
                FalsePosCount+=1
            else:
                FalseNeg=FalseNeg+FeatureData.iat[i,(F5Count+1)]
                FalsePosCount+=1
     
    Total=FalsePosCount+FalsePosCount
    Coeff=FalsePosCount/Total
           
    Eph=(1/(Epsilon[BRound]))
    EphInv=1/(1-Epsilon[BRound])    
    Alpha[BRound]=(1/2)*(math.log((1-Epsilon[BRound])/(Epsilon[BRound])))
    i=0
    hlabel2=1
    
    for i in range(FaceImageListSize):        
        if((FeatureData.iat[i,TopFeature[BRound]])>=(Threshold[BRound])):
            val=1
        else:
            val=-1     

        hlabel2=(val*Polarity[BRound])       
        if(hlabel2!=FeatureData.iat[i,F5Count]):
            if(hlabel2==1):
                FeatureData.iat[i,(F5Count+1)]=(1/2)*(FeatureData.iat[i,(F5Count+1)]*Eph)
            else:
                FeatureData.iat[i,(F5Count+1)]=(1/2)*(FeatureData.iat[i,(F5Count+1)]*Eph)*(1/1-Coeff)
        else:
            FeatureData.iat[i,(F5Count+1)]=(1/2)*(FeatureData.iat[i,(F5Count+1)]*(EphInv))
            
for Rcount in range(1,BoostRounds+1):
    for im in range(FaceImageListSize):
        HSum=0
        for b in range(Rcount):          
            if((FeatureData.iat[im,TopFeature[b]])>=(Threshold[b])):
                val=1
            else:
                val=-1        
            HSum=(Alpha[b]*val*Polarity[b])+HSum                     
                     
        if(HSum>=0):
            FinalDectection=1
        else:
            FinalDectection=-1
            
        if(FinalDectection!=FeatureData.iat[im,F5Count]):
            if(FinalDectection==1):
                RoundFP[Rcount-1]+=1
            else:
                RoundFN[Rcount-1]+=1
        else:
            RoundAccuracy[Rcount-1]+=1
    
    RoundFP[Rcount-1]=(RoundFP[Rcount-1]/FaceImageListSize)
    RoundFN[Rcount-1]=(RoundFN[Rcount-1]/FaceImageListSize)
    RoundAccuracy[Rcount-1]=(RoundAccuracy[Rcount-1]/FaceImageListSize)
    
print("Data for False Negative")         
print("Threshold is :",Threshold)
print("TopFeature is :",TopFeature)
print("RoundAccuracy is :",RoundAccuracy)
print("RoundFP is :",RoundFP)
print("RoundFN is :",RoundFN)

##Getting feature box in image
def getBoxImage(feature,x,y,w,h):
    for i in range(len(image)):
        for j in range(len(image[0])):
            if feature == 1: # 'twoVerticle'
                if i >= y and i < y + h//2 and j >= x and j < x + w:
                    image[i][j] = 255
                elif i >= y + h//2 and i < y + h and j >= x and j < x + w:
                    image[i][j] = 0
            elif feature == 2: #'twoHorizontal'
                if i >= y and i < y + h and j >= x and j < x + w//2:
                    image[i][j] = 0
                elif i >= y and i < y + h and j >= x + w//2 and j < x + w:
                    image[i][j] = 255
            elif feature == 3: #'threeHorizontal'
                if i >= y and i < y + h and j >= x and j < x + w//3:
                    image[i][j] = 0
                elif i >= y and i < y + h and j >= x + w//3 and j < x + w//3 * 2:
                    image[i][j] = 255
                elif i >= y and i < y + h and j >= w//3 * 2 and j < x + w:
                    image[i][j] = 0
            elif feature == 4: #'threeVerticle'
                if i >= y and i < y + h//3 and j >= x and j < x + w:
                    image[i][j] = 0
                elif i >= y + h//3 and y + h//3 * 2  and j >= x and j < x + w:
                    image[i][j] = 255
                elif i >= y + h//3 * 2 and i < y + h and j >= x and j < x + w:
                    image[i][j] = 0
            elif feature == 5: #'four'
                if i >= y and i < y + h//2 and j >= x and j < x + w//2:
                    image[i][j] = 0
                elif i >= y + h//2 and i < y + h and j >= x and j < x + w//2:
                    image[i][j] = 255
                elif i >= y and i < y + h//2 and j >= x + w//2 and j < x + w:
                    image[i][j] = 255
                elif i >= y + h//2 and i < y + h and j >= x + w//2 and j < x + w:
                    image[i][j] = 0
    
img  = Image.open('face00003.png')
imgwidth, imgheight = img.size
image_pixel = np.asarray(img)
image_copy=np.asarray([[0]*imgwidth for i in range(imgheight)])
for i in range(imgheight):
    for j in range(imgwidth):
        image_copy[i][j]=image_pixel[i][j] 
getBoxImage(2, 8, 14, 3,4)
plt.imshow(image_copy)
plt.show()